In [2]:
# ============================================================================
# CELL 1: IMPORTS AND SETUP
# ============================================================================
import lseg.data as rd
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported")

# Set display options
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

✓ Libraries imported


In [3]:
rd.open_session()
print("✓ Refinitiv session opened")

# Simplified universe - most liquid only
universe = {
    'rates': {
        'TYc1': 'US 10Y Note',    # Front month
        'TYc2': 'US 10Y Note 2nd', # Second month (for rolldown)
    },
    'fx': {
        'EUR=': 'EURUSD',
        'JPY=': 'USDJPY',
        'AUD=': 'AUDUSD',
    },
    'equities': {
        '.SPX': 'S&P 500',
        
    },
    'commodities': {
        'CLc1': 'WTI Crude',
        'CLc2': 'WTI Crude 2nd',        
        'GCc1': 'Gold',
        'GCc2': 'Gold 2nd',
    }
}

all_tickers = []
for instruments in universe.values():
    all_tickers.extend(instruments.keys())

print(f"\n✓ Simplified universe: {len(all_tickers)} instruments")
for asset_class, instruments in universe.items():
    print(f"  {asset_class}: {', '.join(instruments.keys())}")


✓ Refinitiv session opened

✓ Simplified universe: 10 instruments
  rates: TYc1, TYc2
  fx: EUR=, JPY=, AUD=
  equities: .SPX
  commodities: CLc1, CLc2, GCc1, GCc2


In [4]:
# ============================================================================
# CELL 3: LOAD DATA (ALL IN ONE DF)
# ============================================================================
start_date = '2019-01-01'
end_date = '2024-12-31'

print(f"\nLoading data from {start_date} to {end_date}...")

#  Load all prices except FX
print("\n1. Loading EQ, Rates and Commods prices...")
prices = rd.get_history(
    universe=all_tickers,
    fields='TRDPRC_1',
    interval='daily',
    start=start_date,
    end=end_date
)

# Drop unwanted FX columns
prices = prices.drop(columns=['JPY=', 'AUD=', 'EUR='], errors='ignore')

#  Load FX prices
fx_tickers = list(universe['fx'].keys())
if fx_tickers:
    try:
        fx_data = rd.get_history(
            universe=fx_tickers,
            fields='BID',
            interval='daily',
            start=start_date,
            end=end_date
        )
        if isinstance(fx_data, pd.DataFrame):
            prices = pd.concat([prices, fx_data], axis=1)
    except Exception as e:
        print(f"   ! Trying CF_CLOSE for FX: {e}")
        try:
            fx_data = rd.get_history(
                universe=fx_tickers,
                fields='CF_CLOSE',
                interval='daily',
                start=start_date,
                end=end_date
            )
            if isinstance(fx_data, pd.DataFrame):
                prices = pd.concat([prices, fx_data], axis=1)
        except Exception as e2:
            print(f"   ✗ Error loading FX: {e2}")

# Load UST yield and duration
print("\n3. Loading UST yield and duration...")
try:
    ust_data = rd.get_history(
        universe='US10YT=RR',
        fields=['MOD_DURTN', 'YLDTOMAT'],
        interval='daily',
        start=start_date,
        end=end_date
    )
    prices = pd.concat([prices, ust_data], axis=1)
except Exception as e:
    print(f"   ! Could not load UST data: {e}")
    # Fallback: add constants
    prices['MOD_DURTN'] = 8.0
    prices['YLDTOMAT'] = 4.0

# Load short-term FX interest rates
print("\n4. Loading short-term FX interest rates...")
try:
    rate_tickers = ['USD3MD=', 'EUR3MD=', 'JPY3MD=', 'AUD3MD=']
    interest_rates = rd.get_history(
        universe=rate_tickers,
        fields='BID',
        interval='daily',
        start=start_date,
        end=end_date
    )
    if isinstance(interest_rates, pd.DataFrame) and not interest_rates.empty:
        prices = pd.concat([prices, interest_rates], axis=1)
except Exception as e:
    print(f"   ! Could not load short-term FX rates: {e}")

# Load equity dividend carry (SPX / SPXTR)
print("\n5. Loading equity dividend carry (SPX)...")
price_data = rd.get_history(
    universe='.SPX',
    fields='TR.PriceClose',
    interval='daily',
    start=start_date,
    end=end_date
)
tr_data = rd.get_history(
    universe='.SPXTR',
    fields='TR.PriceClose',
    interval='daily',
    start=start_date,
    end=end_date
)

# Helper to extract price series
def extract_price(df):
    if isinstance(df.columns, pd.MultiIndex):
        return df.xs('TR.PriceClose', level=1, axis=1).iloc[:, 0]
    else:
        return df.iloc[:, 0]

spx_price = extract_price(price_data)
spxtr_price = extract_price(tr_data)

df = pd.concat([spx_price, spxtr_price], axis=1).dropna()
df.columns = ['SPX', 'SPXTR']

# Dividend return → annualised carry
price_ret = df['SPX'].pct_change()
tr_ret = df['SPXTR'].pct_change()
dividend_return = tr_ret - price_ret
equity_div_carry = dividend_return.rolling(63).mean() * 252 * 100

# Concatenate to prices
prices = pd.concat([prices, equity_div_carry.rename('SPX_div_carry')], axis=1)

print("\n✓ Data loading complete")
print(f"  Prices DF shape: {prices.shape}")
print(f"  Columns sample: {prices.columns}")



Loading data from 2019-01-01 to 2024-12-31...

1. Loading EQ, Rates and Commods prices...

3. Loading UST yield and duration...

4. Loading short-term FX interest rates...

5. Loading equity dividend carry (SPX)...

✓ Data loading complete
  Prices DF shape: (1567, 17)
  Columns sample: Index(['TYc1', 'TYc2', '.SPX', 'CLc1', 'CLc2', 'GCc1', 'GCc2', 'EUR=', 'JPY=', 'AUD=', 'MOD_DURTN', 'YLDTOMAT', 'USD3MD=', 'EUR3MD=', 'JPY3MD=', 'AUD3MD=', 'SPX_div_carry'], dtype='object')


In [5]:
# ============================================================================
# CELL 4: CLEAN DATA
# ============================================================================
print("\n" + "="*60)
print("DATA CLEANING")
print("="*60)

# Check and clean prices
missing_pct = prices.isna().sum() / len(prices) * 100
print(f"\nMissing data: (>5%)")
for col in prices.columns:
    pct = missing_pct[col]
    if pct > 5:
        print(f"  {col}: {pct:.1f}%")

# Keep only assets with <50% missing
prices_clean = prices.loc[:, missing_pct < 50].copy()
prices_clean = prices_clean.fillna(method='ffill').dropna()

print(f"\n✓ Clean prices: {prices_clean.shape}")
print(f"  Assets: {list(prices_clean.columns)}")

# Calculate returns
returns = prices_clean.pct_change().dropna()


print(f"\n✓ Returns: {returns.shape}")
print(f"✓ Carry data cleaned")



DATA CLEANING

Missing data: (>5%)
  TYc2: 5.2%
  GCc1: 8.9%
  SPX_div_carry: 7.6%

✓ Clean prices: (1501, 17)
  Assets: ['TYc1', 'TYc2', '.SPX', 'CLc1', 'CLc2', 'GCc1', 'GCc2', 'EUR=', 'JPY=', 'AUD=', 'MOD_DURTN', 'YLDTOMAT', 'USD3MD=', 'EUR3MD=', 'JPY3MD=', 'AUD3MD=', 'SPX_div_carry']

✓ Returns: (1500, 17)
✓ Carry data cleaned
